# Assignment 2 — Dataset Preprocessing

 ## Goals
 - Explore and clean the dataset.
 - Export JSON files for UI (`site_items.json`, `site_reviews.json`).
 - Create a SQLite database (`app.db`) with two tables:
   - **items**: one row per clothing item.
   - **reviews**: all customer reviews linked to items.

 ---
 ## Pipeline
 1. **Locate data paths** (CSV, DB, JSON).
 2. **Load & normalize dataset** (columns, types).
 3. **Explore summary** (columns, distributions).
 4. **Build items & reviews DataFrames**.
 5. **Export JSON catalog/reviews**.
  6. **Insert into SQLite database**.

## 1. Locate project paths

In [20]:



from pathlib import Path
import pandas as pd
import numpy as np
import sqlite3, json

# Locate CSV robustly
NB_DIR = Path.cwd().resolve()
CANDIDATES = [
    NB_DIR / "data" / "assignment3_II.csv",
    NB_DIR.parent / "data" / "assignment3_II.csv",
    NB_DIR.parents[1] / "data" / "assignment3_II.csv",
]
DATA_CSV = next((p for p in CANDIDATES if p.exists()), None)
if DATA_CSV is None:
    raise FileNotFoundError(" Place 'assignment3_II.csv' under project_root/data/")

PROJECT_ROOT = DATA_CSV.parent.parent
DATA_DIR = PROJECT_ROOT / "data"
DB_PATH = DATA_DIR / "app.db"
CATALOG_JSON = DATA_DIR / "site_items.json"
REVIEWS_JSON = DATA_DIR / "site_reviews.json"

print(" CSV:", DATA_CSV)
print(" DB :", DB_PATH)

 CSV: /Users/mac/Desktop/dem-web/data/assignment3_II.csv
 DB : /Users/mac/Desktop/dem-web/data/app.db


## 2. Load & normalize dataset

In [21]:

df = pd.read_csv(DATA_CSV)

# Expected columns
expected_cols = [
    "Clothing ID","Age","Title","Review Text","Rating",
    "Recommended IND","Positive Feedback Count",
    "Division Name","Department Name","Class Name",
    "Clothes Title","Clothes Description"
]
for col in expected_cols:
    if col not in df.columns:
        df[col] = np.nan

# Type safety
def to_int_safe(x, default=0):
    try: return int(float(x))
    except: return default

for c in ["Clothing ID","Age","Rating","Recommended IND","Positive Feedback Count"]:
    df[c] = df[c].apply(to_int_safe)

for c in ["Title","Review Text","Division Name","Department Name","Class Name","Clothes Title","Clothes Description"]:
    df[c] = df[c].fillna("").astype(str).str.strip()

# Drop invalid Clothing IDs
df = df[df["Clothing ID"] > 0].copy()

print("Loaded:", len(df), "rows")
df.head(3)

Loaded: 19662 rows


,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,Clothes Title,Clothes Description
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,Elegant A-Line Dress,"A classic A-line dress that flows gracefully, ..."
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,Petite High-Waisted Trousers,"Chic, high-waisted trousers designed to elonga..."
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,Silk Button-Up Blouse,A luxurious silk blouse with a timeless button...


## 3. Quick summary of columns

In [22]:

summary = []
for col in df.columns:
    n_unique = df[col].nunique(dropna=True)
    dtype = df[col].dtype

    if dtype in ["int64","float64"]:
        col_type = "Categorical (numeric)" if n_unique <= 10 else "Numerical"
    elif dtype == "object":
        col_type = "Categorical" if n_unique < 50 else "Textual"
    else:
        col_type = "Other"

    summary.append({"Column": col, "Type": col_type, "Unique": n_unique})

pd.DataFrame(summary)

,Column,Type,Unique
0,Clothing ID,Numerical,1095
1,Age,Numerical,77
2,Title,Textual,13983
3,Review Text,Textual,19656
4,Rating,Categorical (numeric),5
5,Recommended IND,Categorical (numeric),2
6,Positive Feedback Count,Numerical,79
7,Division Name,Categorical,3
8,Department Name,Categorical,6
9,Class Name,Categorical,20


## 4. Build `items_df`
 One row per clothing item:
 - Prefer `Clothes Title` → fallback `Title`.
 - Prefer `Clothes Description` → fallback first review text.

In [23]:
def first_non_empty(series: pd.Series) -> str:
    for v in series:
        if isinstance(v, str) and v.strip():
            return v.strip()
    return ""

# Aggregate per Clothing ID (meta only)
agg_meta = (
    df.groupby("Clothing ID", as_index=False)
      .agg({
          "Division Name": first_non_empty,
          "Department Name": first_non_empty,
          "Class Name": first_non_empty,
          "Clothes Title": "first",
          "Clothes Description": "first",
          "Review Text": "first"
      })
)

# Vectorized fallback logic
title_pref = agg_meta["Clothes Title"].fillna("").astype(str).str.strip()
title_fb   = agg_meta["Review Text"].fillna("").astype(str).str.strip()
desc_pref  = agg_meta["Clothes Description"].fillna("").astype(str).str.strip()
desc_fb    = agg_meta["Review Text"].fillna("").astype(str).str.strip().str.slice(0, 200)

agg_meta["clothes_title"] = np.where(title_pref == "", title_fb, title_pref)
agg_meta["description"]   = np.where(desc_pref == "", desc_fb, desc_pref)

# Final items_df
items_df = (
    agg_meta.rename(columns={
        "Clothing ID": "clothing_id",
        "Division Name": "division",
        "Department Name": "department",
        "Class Name": "class_name",
    })[["clothing_id","clothes_title","description","division","department","class_name"]]
)

print(" Items_df built:", items_df.shape)
items_df.head(3)

 Items_df built: (1095, 6)


,clothing_id,clothes_title,description,division,department,class_name
0,1,Layering Bodysuit,A seamless bodysuit perfect for layering. Comf...,Initmates,Intimate,Layering
1,2,Classic Ribbed Knit Top,A versatile ribbed knit top that pairs well wi...,General,Tops,Knits
2,4,Cashmere Turtleneck Sweater,A luxurious cashmere turtleneck that combines ...,General,Tops,Sweaters


## 5. Build reviews_df (all rows)

 Each review will be a row in `reviews` table.
 - Includes metadata: `age`, `title`, `review_text`, `rating`, `recommended`, `positive_feedback`.
 - Clean types (int where needed, safe string for text).
 - Drop invalid rows (`clothing_id <= 0` or empty review_text).

In [24]:
reviews_df = df[[
    "Clothing ID","Age","Title","Review Text","Rating",
    "Recommended IND","Positive Feedback Count"
]].copy()

reviews_df.rename(columns={
    "Clothing ID": "clothing_id",
    "Age": "age",
    "Title": "title",
    "Review Text": "review_text",
    "Rating": "rating",
    "Recommended IND": "recommended",
    "Positive Feedback Count": "positive_feedback",
}, inplace=True)

# Safe type coercion
def to_int_safe(x, default=0):
    try: return int(float(x))
    except: return default

for col in ["clothing_id","age","rating","positive_feedback"]:
    reviews_df[col] = reviews_df[col].apply(to_int_safe)

reviews_df["recommended"] = reviews_df["recommended"].apply(lambda x: 1 if to_int_safe(x) == 1 else 0)
reviews_df["title"]       = reviews_df["title"].fillna("").astype(str).str.strip()
reviews_df["review_text"] = reviews_df["review_text"].fillna("").astype(str).str.strip()

# Drop invalids
reviews_df = reviews_df[(reviews_df["clothing_id"] > 0) & (reviews_df["review_text"].str.len() > 0)]
reviews_df = reviews_df[reviews_df["rating"].between(1,5)]

print("Reviews_df built:", reviews_df.shape)
reviews_df.head(3)

Reviews_df built: (19662, 7)


,clothing_id,age,title,review_text,rating,recommended,positive_feedback
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6


## 6. Export JSON files
- `site_items.json`: catalog for index page.
 - `site_reviews.json`: all reviews.

In [25]:

CATALOG_JSON.write_text(items_df.to_json(orient="records", indent=2), encoding="utf-8")
REVIEWS_JSON.write_text(reviews_df.to_json(orient="records", indent=2), encoding="utf-8")

print( CATALOG_JSON)
print( REVIEWS_JSON)

/Users/mac/Desktop/dem-web/data/site_items.json
/Users/mac/Desktop/dem-web/data/site_reviews.json


## 7. Create SQLite DB
 Two tables:
 - `items(clothing_id PK, clothes_title, description, division, department, class_name)`
 - `reviews(id PK, clothing_id FK, age, title, review_text, rating, recommended, positive_feedback)`

In [26]:

con = sqlite3.connect(DB_PATH)
cur = con.cursor()
cur.executescript("""
DROP TABLE IF EXISTS reviews;
DROP TABLE IF EXISTS items;

CREATE TABLE items (
    clothing_id   INTEGER PRIMARY KEY,
    clothes_title TEXT,
    description   TEXT,
    division      TEXT,
    department    TEXT,
    class_name    TEXT
);

CREATE TABLE reviews (
    id                INTEGER PRIMARY KEY AUTOINCREMENT,
    clothing_id       INTEGER NOT NULL,
    age               INTEGER,
    title             TEXT,
    review_text       TEXT,
    rating            INTEGER,
    recommended       INTEGER,
    positive_feedback INTEGER,
    FOREIGN KEY (clothing_id) REFERENCES items (clothing_id) ON DELETE CASCADE
);
""")

cur.executemany("""
INSERT INTO items (clothing_id, clothes_title, description, division, department, class_name)
VALUES (?,?,?,?,?,?)
""", items_df.itertuples(index=False, name=None))

cur.executemany("""
INSERT INTO reviews (clothing_id, age, title, review_text, rating, recommended, positive_feedback)
VALUES (?,?,?,?,?,?,?)
""", reviews_df.itertuples(index=False, name=None))

con.commit()
print("✅ Inserted", len(items_df), "items and", len(reviews_df), "reviews.")
con.close()

✅ Inserted 1095 items and 19662 reviews.
